In [1]:
import twstock
from twstock import stock
import sqlite3
import pandas as pd
from datetime import datetime
import os

In [2]:
strdate = datetime.now().strftime("%Y%m%d")

In [3]:
def save_stock(sid:str,strdate=None,enddate=None):
    stock_data = stock.Stock(sid)

    if strdate == None and enddate == None:
        strdate = datetime.now().strftime("%Y-%m-%d")
    elif enddate == None:
        year = int(strdate[0:4]) ; month = int(strdate[5:7])
        stock_data.fetch(year=year,month=month)
    else:
        syear = int(strdate[0:4]) ; smonth = int(strdate[5:7])
        eyear = int(enddate[0:4]) ; emonth = int(enddate[5:7])
        stock_data.fetch_from_to(syear=syear,smonth=smonth,eyear=eyear,emonth=emonth)

    dbdir = 'stocksdb'
    dbname = '%s/%s.db' % (dbdir,sid)
    try:
        os.mkdir(dbdir)
    except:        
        pass
    
    conn = sqlite3.connect(dbname,detect_types=sqlite3.PARSE_DECLTYPES)
    cursor = conn.cursor()
 
    # Create table
    cursor.execute('''CREATE TABLE IF NOT EXISTS stocks
        (date timestamp, capacity integer, turnover text, open real, high real, 
        low real, close real, change real, transactions integer)''')

    # Creat unique index
    cursor.execute('CREATE UNIQUE INDEX IF NOT EXISTS date_unique ON stocks (date)')

    # Insert a row of data
    for data in stock_data.data:
        cursor.execute("INSERT OR IGNORE INTO stocks VALUES (?,?,?,?,?,?,?,?,?)",data)

    # Save (commit) the changes
    conn.commit()
 
    # We can also close the connection if we are done with it.
    # Just be sure any changes have been committed or they will be lost.
    conn.close()

In [4]:
def read_stock(sid:str,strdate=None,enddate=None):
    if enddate == None:
        enddate = datetime.now().strftime("%Y-%m-%d")

    dbdir = 'stocksdb'
    dbname = '%s/%s.db' % (dbdir,sid)
    conn = sqlite3.connect(dbname,detect_types=sqlite3.PARSE_DECLTYPES)
    cursor = conn.cursor()

    # Read table
    sqlite_data = cursor.execute('''SELECT * FROM stocks WHERE date >= datetime(?) AND date <= datetime(?)''',(strdate,enddate))
    
    data_pd = pd.DataFrame(sqlite_data,columns=['date', 'capacity', 'turnover', 'open', 'high', 'low', 'close', 'change', 'transaction'])
    
    return data_pd

In [5]:
stocklist =['2345','2405','2499','2610']

In [8]:
save_stock('2345','2019-03-01','2019-05-01')

In [ ]:
read_stock('2345','2019-03-01','2019-04-15')

# 收集所有的股號

In [ ]:
def get_stockids():
    sids = []
    twse = twstock.twse
    for sid in twse.keys():
        if twse[sid].type == '股票':
            sids.append(sid)
    return sids

# 儲存最近日期之股市資料至sqlite

In [ ]:
sids = get_stockids()
for sid in sids:
    print('Downloading ...%5s'%(sid))
    try:
        save_stock(sid)
    except:
        print(twse[sid].name,sid,' Calculate failed')

# 儲存指定日期之股市資料至sqlite

In [ ]:
strdate='2018-08-01'
sids = get_stockids()
for sid in sids:
    print('Downloading ...%5s'%(sid))
    try:
        save_stock(sid,strdate)
    except:
        print(twse[sid].name,sid,' Calculate failed')